In [25]:
import pandas as pd
import numpy as np
import string
import re
import random 
import os
import csv

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import metrics
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline


from imblearn.ensemble import BalancedRandomForestClassifier

import gensim
import gensim.downloader as gensim_api
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

# Tf-idf RF


In [28]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220115_all_paragraphs_2020_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_high_pride.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # pipeline BOW, tfidf and RF
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier(n_estimators=5000, max_depth=3,max_features='sqrt', random_state=0, class_weight="balanced")),
                                                 ])
      ## Fit the model to the training set
    text_clf = text_clf.fit(train['text'], train['pred_class'])
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test['text'])
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(test['true_class'], preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(test['true_class'],  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(test['true_class'],text_clf.predict(test['text']) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.8994561328180908
PR AUC: 0.46307102434292435
AUC: 0.8526549305853729
PR AUC: 0.4114321821991345
AUC: 0.8738371260913125
PR AUC: 0.48183202495918653
AUC: 0.8761270931730355
PR AUC: 0.46443772592711435
AUC: 0.8594532703592386
PR AUC: 0.5062518174590759
AUC: 0.7774748923959828
PR AUC: 0.32999202276437317
AUC: 0.833476456275941
PR AUC: 0.392916544318665
AUC: 0.8189494776012596
PR AUC: 0.3046579340549502
AUC: 0.8713324745956776
PR AUC: 0.44520121719302047
AUC: 0.8580220409331615
PR AUC: 0.40510400875135855
AUC: 0.8822098182338629
PR AUC: 0.5260744227138304
AUC: 0.8799198511521397
PR AUC: 0.44511781360051894
AUC: 0.853084299413196
PR AUC: 0.3846262563327213
AUC: 0.856312769010043
PR AUC: 0.49336092122129793
AUC: 0.8800629740947474
PR AUC: 0.4776010258230655
AUC: 0.8623157292113925
PR AUC: 0.320394045041141
AUC: 0.8837159253945481
PR AUC: 0.4591976122651373
AUC: 0.841133533705453
PR AUC: 0.3202335894521821
AUC: 0.8524402461714613
PR AUC: 0.38175511429046627
AUC: 0.8757692858165164
PR A

In [29]:
with open("high_pride_pr_auc_tfidf.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("high_pride_auc_tfidf.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Local Word2Vec RF

In [30]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220115_all_paragraphs_2020_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_high_pride_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")
avg_embeddings = loadtxt('avg_embeddings.csv', delimiter=',')

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # attach train/test set to original df
    train_id = train['speech_par_id']
    test_id = test['speech_par_id']    
    dat['test'] = np.where(dat['speech_par_id'].isin(test_id), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train_id), 1, 0)
    # get the embeddings to train/test set
    train_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(train_id)].index.tolist()])
    test_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(test_id)].index.tolist()])
    ## Fit the model to the training set
    text_clf = RandomForestClassifier(n_estimators=500, max_depth=3, random_state=0, max_features = "sqrt", class_weight="balanced")
    text_clf.fit(train_vec,  dat[dat['train']==1].pred_class) # use the original df to match the order
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test_vec)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class,text_clf.predict(test_vec) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.8180057388809182
PR AUC: 0.442222635602635
AUC: 0.8570917418062115
PR AUC: 0.34507768364641117
AUC: 0.8300415056533562
PR AUC: 0.39752272300070157
AUC: 0.8600973236009732
PR AUC: 0.41467470069598406
AUC: 0.8236725347073136
PR AUC: 0.34746236432557553
AUC: 0.849713055954089
PR AUC: 0.37704024329418273
AUC: 0.8079626972740316
PR AUC: 0.35298389224376986
AUC: 0.8096464863317591
PR AUC: 0.3420993310852235
AUC: 0.8178760555317017
PR AUC: 0.44885006048891235
AUC: 0.7893830703012913
PR AUC: 0.34242156327699774
AUC: 0.795334192070989
PR AUC: 0.295998945455875
AUC: 0.8877200515242594
PR AUC: 0.5196919017352609
AUC: 0.7565997130559541
PR AUC: 0.2773729157310143
AUC: 0.8600430416068867
PR AUC: 0.3711884609467261
AUC: 0.8436381852010877
PR AUC: 0.3651305209372267
AUC: 0.7914698726205811
PR AUC: 0.2853724949695878
AUC: 0.8241750358680058
PR AUC: 0.33267491714530806
AUC: 0.823385588954408
PR AUC: 0.3889122795607916
AUC: 0.7928295405753543
PR AUC: 0.2652908992060442
AUC: 0.8725490196078431
PR 

In [31]:
with open("high_pride_pr_auc_w2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("high_pride_auc_w2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Pretrained Word2Vec

In [32]:
auc_scores = []
pr_auc = []

dat = pd.read_csv("20220115_all_paragraphs_2020_added.csv")
annotated = pd.read_csv("All_annotated_data_round_1_high_pride_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")
avg_embeddings = loadtxt('avg_embeddings_pretrained.csv', delimiter=',')

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # attach train/test set to original df
    train_id = train['speech_par_id']
    test_id = test['speech_par_id']    
    dat['test'] = np.where(dat['speech_par_id'].isin(test_id), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train_id), 1, 0)
    # get the embeddings to train/test set
    train_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(train_id)].index.tolist()])
    test_vec = np.asarray([avg_embeddings[i] for i in dat[dat['speech_par_id'].isin(test_id)].index.tolist()])
    ## Fit the model to the training set
    text_clf = RandomForestClassifier(n_estimators=500, max_depth=3, random_state=0, max_features = "sqrt", class_weight="balanced")
    text_clf.fit(train_vec,  dat[dat['train']==1].pred_class) # use the original df to match the order
    ## Predict out-of-sample on the test set and compute AUC
    preds = text_clf.predict_proba(test_vec)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores = auc_scores + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    print("PR AUC: " + str(lr_auc))
    
print("Mean AUC: " + str(np.mean(auc_scores)))
print("Mean PR AUC: " + str(np.mean(pr_auc)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class,text_clf.predict(test_vec) )
print(confusion)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AUC: 0.8251793400286944
PR AUC: 0.40192543580128953
AUC: 0.8314727350794332
PR AUC: 0.37246305584621925
AUC: 0.8424932016602261
PR AUC: 0.3431718278291743
AUC: 0.8542292829540575
PR AUC: 0.4234265759811332
AUC: 0.8577357950479462
PR AUC: 0.3849908788034974
AUC: 0.8949784791965566
PR AUC: 0.46788137908162664
AUC: 0.8158536585365854
PR AUC: 0.38359594896684096
AUC: 0.8550164591383999
PR AUC: 0.4372844785920216
AUC: 0.8276799771003293
PR AUC: 0.42025494770499255
AUC: 0.8258967001434719
PR AUC: 0.35323759117011805
AUC: 0.8780592528982396
PR AUC: 0.3999754712374212
AUC: 0.9028195219693718
PR AUC: 0.4792163744768058
AUC: 0.784648493543759
PR AUC: 0.2905315955999843
AUC: 0.8585365853658536
PR AUC: 0.32712559668590907
AUC: 0.8442106769715185
PR AUC: 0.3137022509185967
AUC: 0.8447831687419494
PR AUC: 0.3136624145143112
AUC: 0.8543758967001435
PR AUC: 0.3036514781879088
AUC: 0.8208686897741982
PR AUC: 0.38049962288493433
AUC: 0.7941176470588235
PR AUC: 0.3207199906245569
AUC: 0.8623872906826964


In [33]:
with open("high_pride_pr_auc_pretrained.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("high_pride_auc_pretrained.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores:
        csvwriter.writerow([row])

# Doc2Vec RF

In [35]:
auc_scores_d2v = []
pr_auc = []
accuracy_scores_d2v = []

dat = pd.read_csv("20201115_all_paragraphs.csv")
dat['speech_par_id'] = dat['Speech_id'].astype(str) + "_" + dat['par_id'].astype(str)

model = Word2Vec.load("doc2vec_wordvecs.model") 
annotated = pd.read_csv("All_annotated_data_round_1_high_pride_20220209.csv")
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")

for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/"):
    ## load the test set 
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # merge test and train sets back onto the total df
    dat['test'] = np.where(dat['speech_par_id'].isin(test["speech_par_id"]), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train["speech_par_id"]), 1, 0)
    # so that teh order of rows are the same with the embeddings
    test_set = np.asarray([model.docvecs[i] for i in dat[dat['test'] == 1].index.tolist()])
    train_set = np.asarray([model.docvecs[i] for i in dat[dat['train']== 1].index.tolist()])
    ## Initialize a random forest classifier
    gbc = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=0, max_features = "sqrt", class_weight="balanced")
    ## Fit the model to the training set
    gbc.fit(train_set, dat[dat['train']==1].pred_class)
    ## Predict out-of-sample on the test set and compute AUC
    preds = gbc.predict_proba(test_set)
    fpr_d2v, tpr_d2v, thresholds_d2v = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores_d2v = auc_scores_d2v + [metrics.auc(fpr_d2v, tpr_d2v)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc = pr_auc + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr_d2v, tpr_d2v)))
    accuracy_d2v = metrics.accuracy_score(dat[dat['test']==1].pred_class, gbc.predict(test_set))
    accuracy_scores_d2v = accuracy_scores_d2v + [accuracy_d2v]
    print("Accuracy: " + str(accuracy_d2v))
    print("PR_AUC: " + str(lr_auc))

print("Mean AUC: " + str(np.mean(auc_scores_d2v)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_d2v)))

confusion = confusion_matrix(dat[dat['test']==1].pred_class, gbc.predict(test_set))
print(confusion)

AUC: 0.835466705019416
Accuracy: 0.9232505643340858
PR_AUC: 0.39994181156287323
AUC: 0.8451219512195122
Accuracy: 0.9234234234234234
PR_AUC: 0.3055014082745046
AUC: 0.8227403156384504
Accuracy: 0.9256756756756757
PR_AUC: 0.41585992996961074
AUC: 0.8711621233859397
Accuracy: 0.9234234234234234
PR_AUC: 0.4590812155542532
AUC: 0.8052812365822241
Accuracy: 0.9235955056179775
PR_AUC: 0.3089456809181288
AUC: 0.8645189126995542
Accuracy: 0.9255079006772009
PR_AUC: 0.49701786180441687
AUC: 0.8701999137063138
Accuracy: 0.9232505643340858
PR_AUC: 0.44378342556800904
AUC: 0.8328324030342064
Accuracy: 0.9235955056179775
PR_AUC: 0.37137146948231115
AUC: 0.8727637040217546
Accuracy: 0.9235955056179775
PR_AUC: 0.42687718477745434
AUC: 0.840028694404591
Accuracy: 0.9234234234234234
PR_AUC: 0.3625260915100735
AUC: 0.8632245074068746
Accuracy: 0.9232505643340858
PR_AUC: 0.40579683227609853
AUC: 0.8890081580077286
Accuracy: 0.9235955056179775
PR_AUC: 0.4352501896973803
AUC: 0.7949067431850789
Accuracy: 0

In [37]:
with open("high_pride_pr_auc_d2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc:
        csvwriter.writerow([row])    
    
with open("high_pride_auc_d2v.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores_d2v:
        csvwriter.writerow([row])

In [36]:
print("Mean ROC-AUC: " + str(np.mean(auc_scores_d2v)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_d2v)))
print("Mean PR_AUC: " + str(np.mean(pr_auc)))


Mean ROC-AUC: 0.8569520578279224
Mean Accuracy: 0.9236310913011144
Mean PR_AUC: 0.39892911164012196


# balanced RFC

In [38]:
auc_scores_balanced = []
pr_auc_balanced = []
accuracy_scores_balanced = []

dat = pd.read_csv("20201115_all_paragraphs.csv")
dat['speech_par_id'] = dat['Speech_id'].astype(str) + "_" + dat['par_id'].astype(str)
# X = np.asarray([model.docvecs[i] for i in label_dat.index.tolist()])
# Y = np.asarray(label_dat['label'].tolist(), dtype="int")
model = Word2Vec.load("doc2vec_wordvecs.model") 
dat = dat.merge(annotated[["speech_par_id", "pred_class"]], how = "left", on = "speech_par_id")


for filename in os.listdir("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/"):
    ## load the test set 
    annotated = pd.read_csv("All_annotated_data_round_1_high_pride_20220209.csv")
    file = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/codes/Prediction_of_test_high_pride/" + "/" + filename
    test = pd.read_csv(file)
    train = annotated[~annotated['speech_par_id'].isin(test["speech_par_id"])]
    test = test[test['speech_par_id'].isin(dat['speech_par_id'])]
    train = train[train['speech_par_id'].isin(dat['speech_par_id'])] # make sure annotated data and the original df match
    # merge test and train sets back onto the total df
    dat['test'] = np.where(dat['speech_par_id'].isin(test["speech_par_id"]), 1, 0)
    dat['train'] = np.where(dat['speech_par_id'].isin(train["speech_par_id"]), 1, 0)
    # so that teh order of rows are the same with the embeddings
    test_set = np.asarray([model.docvecs[i] for i in dat[dat['test'] == 1].index.tolist()])
    train_set = np.asarray([model.docvecs[i] for i in dat[dat['train']== 1].index.tolist()])
    ## Initialize a random forest classifier
    brfc = BalancedRandomForestClassifier(n_estimators=5000, max_depth=3, random_state=0, max_features = "sqrt")
    ## Fit the model to the training set
    brfc.fit(train_set, dat[dat['train']==1].pred_class)
    ## Predict out-of-sample on the test set and compute AUC
    preds = brfc.predict_proba(test_set)
    fpr, tpr, thresholds = metrics.roc_curve(dat[dat['test']==1].pred_class, preds[:,1], pos_label=1)
    auc_scores_balanced = auc_scores_balanced + [metrics.auc(fpr, tpr)]
    #PR AUC
    lr_precision, lr_recall, _ = precision_recall_curve(dat[dat['test']==1].pred_class,  preds[:,1].tolist())
    lr_auc = auc(lr_recall, lr_precision)
    pr_auc_balanced = pr_auc_balanced + [lr_auc]
    print("AUC: "+str(metrics.auc(fpr, tpr)))
    accuracy_balanced = metrics.accuracy_score(dat[dat['test']==1].pred_class, brfc.predict(test_set))
    accuracy_scores_balanced = accuracy_scores_balanced + [accuracy_balanced]
    print("Accuracy: " + str(accuracy_balanced))
    print("PR_AUC: " + str(lr_auc))

print("Mean AUC: " + str(np.mean(auc_scores_balanced)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_balanced)))

AUC: 0.8607076082266648
Accuracy: 0.7923250564334086
PR_AUC: 0.4817791637319441
AUC: 0.8456241032998565
Accuracy: 0.7747747747747747
PR_AUC: 0.323554349948828
AUC: 0.849426111908178
Accuracy: 0.7725225225225225
PR_AUC: 0.4915918237882323
AUC: 0.8779770444763271
Accuracy: 0.786036036036036
PR_AUC: 0.4262213918082809
AUC: 0.837770144554172
Accuracy: 0.7640449438202247
PR_AUC: 0.39071580318101157
AUC: 0.8850855745721271
Accuracy: 0.8171557562076749
PR_AUC: 0.504705332103836
AUC: 0.8691931540342299
Accuracy: 0.7404063205417607
PR_AUC: 0.49366847079231163
AUC: 0.8465722055245455
Accuracy: 0.8067415730337079
PR_AUC: 0.3902174669453447
AUC: 0.8661800486618004
Accuracy: 0.8157303370786517
PR_AUC: 0.4859437738085214
AUC: 0.8454088952654232
Accuracy: 0.795045045045045
PR_AUC: 0.43171761563685407
AUC: 0.8839349920897455
Accuracy: 0.7562076749435666
PR_AUC: 0.4054418670851804
AUC: 0.90353513668241
Accuracy: 0.7955056179775281
PR_AUC: 0.5234028755985269
AUC: 0.798206599713056
Accuracy: 0.7522522522

In [40]:
with open("high_pride_pr_auc_balanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in pr_auc_balanced:
        csvwriter.writerow([row])    
    
with open("high_pride_auc_balanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in auc_scores_balanced:
        csvwriter.writerow([row])

In [39]:
print("Mean ROC-AUC: " + str(np.mean(auc_scores_balanced)))
print("Mean Accuracy: " + str(np.mean(accuracy_scores_balanced)))
print("Mean PR_AUC: " + str(np.mean(pr_auc_balanced)))

Mean ROC-AUC: 0.8669529043874283
Mean Accuracy: 0.7789894338080665
Mean PR_AUC: 0.42820777920392755
